In [1]:
import pandas as pd
import numpy as np
import ast
#import sklearn.feature_extraction.text as sk_f
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem.porter as nl
import sklearn.metrics.pairwise as sk_m
import difflib

In [2]:
def convert(row):                        #ast.literal_eval--to convert a row from string to list
    l=[]                                 #fn to take name from list and append to another list
    for i in ast.literal_eval(row):
        l.append(i['name'])
    return l    

In [3]:
def convert_cast(row):            #fn to take name from list and append to another list for first 3 elements
    l=[]
    num=0
    for i in ast.literal_eval(row):
        if num!=3:
            l.append(i['name'])
            num+=1
        else:
            break
    return l   

In [4]:
def director(row):            #fn to take name from list and append to another list for first 3 elements
    l=[]
    for i in ast.literal_eval(row):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l  

In [5]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [6]:
movies=movies.merge(credits,on='title')    #merging both csv

In [7]:
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [8]:
movies=movies[['id','title','overview','genres','keywords','cast','crew','vote_average','vote_count']]


In [10]:
movies.dropna(inplace =True)     #remove row with empty data
movies.drop_duplicates()         #return dataframe with duplicate rows removed

,id,title,overview,genres,keywords,cast,crew,vote_average,vote_count
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",7.2,11800
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",6.9,4500
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",6.3,4466
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",7.6,9106
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",6.1,2124
...,...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...","[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...",6.6,238
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...",5.9,5
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...",7.0,6
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,[],[],"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",5.7,7


In [11]:
movies['genres']=movies['genres'].apply(convert)
movies['keywords']=movies['keywords'].apply(convert)
movies['cast']=movies['cast'].apply(convert_cast)
movies['overview']=movies['overview'].apply(lambda x:x.split())   #create a list from string


In [12]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])         #removing space from list elements
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['overview']=movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [13]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']   #creating a row and adding elements from rest of column
movies=movies[['id','title','tags','vote_average','vote_count']]     #taking specific columns and making dataframe

In [15]:
def stem(tag):              #fn to remove similiar words
    l=[]
    for i in tag.split():
        l.append(nl.PorterStemmer().stem(i))
    return " ".join(l)   

In [16]:
movies['tags']=movies['tags'].apply(lambda x:" ".join(x))   #list to string
movies['tags']=movies['tags'].apply(lambda x:x.lower())
movies['tags']=movies['tags'].apply(stem)

In [18]:
tfidf = TfidfVectorizer(stop_words='english')
vectors = tfidf.fit_transform(movies['tags'])

In [19]:
similarity=sk_m.cosine_similarity(vectors)       #compare movies

In [20]:
print(similarity)

[[1.         0.01540111 0.02032658 ... 0.01516159 0.         0.        ]
 [0.01540111 1.         0.00932395 ... 0.01199958 0.         0.00460499]
 [0.02032658 0.00932395 1.         ... 0.010999   0.         0.        ]
 ...
 [0.01516159 0.01199958 0.010999   ... 1.         0.01322439 0.0191042 ]
 [0.         0.         0.         ... 0.01322439 1.         0.01422447]
 [0.         0.00460499 0.         ... 0.0191042  0.01422447 1.        ]]


In [21]:
list_title=movies['title'].tolist()    #making list of all titles

In [23]:
def recommends(movie_name):               #recommend movies
    n_movies_to_reccomend = 100
    close_match=difflib.get_close_matches(movie_name,list_title)   #checking for a close match of movie's title
    movie_index=movies[movies['title']==close_match[0]].index[0]
    dist=similarity[movie_index]
    movies_recommend=(sorted(list(enumerate(dist)),reverse=True,key=lambda x:x[1]))[1:n_movies_to_reccomend]   #creating tuple of similarity score with index and sorting them in decreasing order 
    recommend_frame = []

    for i in movies_recommend:
        recommend_frame.append({'Title':movies.iloc[i[0]].title,'Similarity':i[1]})
    df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend))
    return df

In [25]:
recommends('Iron man').head(15)

,Title,Similarity
1,Iron Man 3,0.337871
2,Iron Man 2,0.328790
3,Avengers: Age of Ultron,0.209311
4,The Avengers,0.155751
5,Captain America: Civil War,0.154447
6,The Incredible Hulk,0.137290
7,Ant-Man,0.128429
8,X-Men,0.122890
9,X-Men Origins: Wolverine,0.117884
10,Thor: The Dark World,0.111318
